<a href="https://colab.research.google.com/github/marcosrabaioli/django-rest/blob/master/Deep_Reiforcement_Learning_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook é baseado no tutorial disponivel em:
- https://towardsdatascience.com/deep-q-network-dqn-i-bce08bdf2af

- https://github.com/jorditorresBCN/Deep-Reinforcement-Learning-Explained/blob/master/DRL_15_16_17_DQN_Pong.ipynb

In [ ]:
import gym
import gym.spaces

# nome do jogo
DEFAULT_ENV_NAME = "PongNoFrameskip-v4" 
test_env = gym.make(DEFAULT_ENV_NAME)
# espaço de acoes possiveis
print(test_env.action_space.n)

In [ ]:
# Descricao das acoes do jogo
# NOOP = FIRE, RIGTH = RIGTHFIRE, LEFT = LEFTFIRE
#   PARADO        DIREITA            ESQUERDA
print(test_env.unwrapped.get_action_meanings())

In [ ]:
# Espaço dimenssional do ambiente (neste caso é o tamanho da imagem do atari)
print(test_env.observation_space.shape)

In [ ]:
# definindo o hadware
!nvidia-smi 

Criando algumas funçoes para iniciar o jogo, reiniciar e tomar açoes

In [ ]:
import cv2
import numpy as np
import collections

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs

Vamos criar uma classe para diminuir a amostragem da tela, por questoes de perfrmance é melhor dividirmos por 4 a amostragem, pois é mais simples e efetivo tomar uma decição a cada 4 quadros. Para uma rede neural seria muito custoso tomar uma decisão a cada quadro e pouco acrescentaria na escolha da ação pois a mudança de um quadro para o outro é minima. Existe tbm uma variabilidade na posição dos pixels de um quadro ímpar para um quadro par, entao amostramos os ultimos 2 pixels e extraimos o max deles

In [ ]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs

Vamos agora criar uma classe que reduz o tamanho da imagem de 210x160 rgb para uma escala de 84x84 em escala de cinza

In [ ]:
class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)

Precisamos obedecer a propeidade de markov para poder implementar um agente. Como precisamos saber a direção da bolinha, e com somente um frame não conseguimos saber, vamos criar um estado que contenha os últimos 4 frames (estados) (ja passado pelo processo de reamostragem da classe MaxAndSkipEnv)

In [ ]:
class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer

Vamos criar uma classe para mudar o formato de entrada para o formato do pytorch de
HWC (height, width, channel) para CHW (channel, height, width) format required by PyTorch. Vamos tbm criar uma classe para normalizar a entrada.

In [ ]:
class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], 
                                old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

Agora vamos criar uma função que inicializa o ambiente

In [ ]:
def make_env(env_name):
    env = gym.make(env_name)
    env = MaxAndSkipEnv(env)
    env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    return ScaledFloatFrame(env)

Agora vamos ao modelo da rede de Deep Q-Learning

In [ ]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package

device = torch.device("cuda")

Primeira camada convolucional:

- As imagens da tela são primeiro processadas por três camadas convolucionais. Isso permite que o sistema explore as relações espaciais e pode explorar o espaço de regra espacial. Além disso, como quatro quadros são empilhados e fornecidos como entrada, essas camadas convolucionais também extraem algumas propriedades temporais entre esses quadros
- As camadas convolucionais são seguidas por uma camada oculta totalmente conectada com ativação ReLU e uma camada de saída linear totalmente conectada que produziu o vetor de valores de ação:

In [ ]:
# Taken from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/dqn_model.py

import numpy as np

class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        # Primeira camada convolucional
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        # Segunda camada full connected
        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)

In [ ]:
test_env = make_env(DEFAULT_ENV_NAME)
test_net = DQN(test_env.observation_space.shape, test_env.action_space.n).to(device)
print(test_net)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

- MEAN_REWARD_BOUND: o limite de recompensa para parar de treinar. Vamos considerar que o jogo convergiu quando o nosso agente atingir uma média de 19 jogos ganhos (de 21) nos últimos 100 jogos
- gamma: o fator de desconto
- batch_size: o tamanho do minibatch no experience replay
- learning_rate: é a taxa de aprendizagem
- replay_size: tamanho do buffer de reprodução (número máximo de experiências armazenadas na memória de reprodução)
- sync_target_frames: indica a frequência com que sincronizamos pesos de modelo da rede DQN principal para a rede DQN target network (quantos quadros entre a sincronização)
- replay_start_size: a contagem de quadros (experiências) para adicionar ao buffer de reprodução antes de iniciar o treinamento.

In [ ]:
import time
import numpy as np
import collections

# Limite de recompensa para parar o treinamento
MEAN_REWARD_BOUND = 19.0           

gamma = 0.99                   
batch_size = 32                
replay_size = 10000            
learning_rate = 1e-4           
sync_target_frames = 1000      
replay_start_size = 10000      

eps_start=1.0
eps_decay=.999985
eps_min=0.02

**Experience Replay**: a ideia básica por trás da repetição da experiência é armazenar experiências passadas e, em seguida, usar um subconjunto aleatório dessas experiências para atualizar a Q-network, em vez de usar apenas a experiência mais recente.

In [ ]:
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceReplay:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)


In [ ]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):

        done_reward = None
        # escolhe uma ação aleatoria com um probabilidade epsilon
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        # escolhe uma ação definida pela politica com probabilidade 1-epsilon
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())
        # Verifica o próximo estado e a recompensa
        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [ ]:
import datetime
print(">>>Training starts at ",datetime.datetime.now())

In [ ]:
env = make_env(DEFAULT_ENV_NAME)

net = DQN(env.observation_space.shape, env.action_space.n).to(device)
target_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
 
buffer = ExperienceReplay(replay_size)
agent = Agent(env, buffer)

epsilon = eps_start

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
total_rewards = []
frame_idx = 0  

best_mean_reward = None

while True:
        frame_idx += 1
        # Atualizando a probabilidade de escolha de ação aleatória
        epsilon = max(epsilon*eps_decay, eps_min)

        # Usando a rede para selecionar e fazer a ação
        reward = agent.play_step(net, epsilon, device=device)
        # Reward é somente diferente de nulo quando um jogo acaba
        if reward is not None:
            total_rewards.append(reward)
            
            # Caculando a media de pontos dos ultimos 100 jogos
            mean_reward = np.mean(total_rewards[-100:])

            print("%d:  %d games, mean reward %.3f, (epsilon %.2f)" % (
                frame_idx, len(total_rewards), mean_reward, epsilon))
            
            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("reward_100", mean_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)

            # Atualizando o melher score e salvando o modelo de melhor score
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
                best_mean_reward = mean_reward
                if best_mean_reward is not None:
                    print("Best mean reward updated %.3f" % (best_mean_reward))

            # Critério de parada do aprendizado
            if mean_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        # Verifica se ja tem experiencia suficiente para usar no experience replay
        # O processo de aprendizado somente começa depois de o agente ter explorado
        # o ambiente por algumas vezes, definido por replay_start_size
        if len(buffer) < replay_start_size:
            continue
        # Buscando na memoria algumas experiencias passadas
        batch = buffer.sample(batch_size)
        states, actions, rewards, dones, next_states = batch

        # Formatação de dados para pytorch
        states_v = torch.tensor(states).to(device)
        next_states_v = torch.tensor(next_states).to(device)
        actions_v = torch.tensor(actions).to(device)
        rewards_v = torch.tensor(rewards).to(device)
        done_mask = torch.ByteTensor(dones).to(device)

        # Obtendo a predição pela funçao Q para cada estado e ação dada (state-action values)
        state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

        # Predizendo o valor da recompensa da melhor ação para o próximo estado 
        next_state_values = target_net(next_states_v).max(1)[0]

        # Se o estado for o último então nao ha recompensa
        next_state_values[done_mask] = 0.0

        # fazendo uma cópia da variavel para evitar problemas de concorrencia de dados
        next_state_values = next_state_values.detach()
        
        # calculando o valor da recompensa esperado (predicao da prox recompensa + recompensa dada pela experiencia)
        expected_state_action_values = next_state_values * gamma + rewards_v

        
        loss_t = nn.MSELoss()(state_action_values, expected_state_action_values)

        optimizer.zero_grad()
        loss_t.backward()
        optimizer.step()

        # Atualiza a target network após cada passo de sincronização
        if frame_idx % sync_target_frames == 0:
            target_net.load_state_dict(net.state_dict())
       
writer.close()

        


In [ ]:
print(">>>Training ends at ",datetime.datetime.now())

In [ ]:
tensorboard  --logdir=runs

In [ ]:
import gym
import time
import numpy as np

import torch

import collections

DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
FPS = 25

Alguns passos para pode exibir o resultado em um jogo aqui no colab

In [ ]:
# Taken from 
# https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7

!apt-get install -y xvfb x11-utils

!pip install pyvirtualdisplay==0.2.* \
             PyOpenGL==3.1.* \
             PyOpenGL-accelerate==3.1.*

!pip install gym[box2d]==0.17.*

import pyvirtualdisplay

_display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
_ = _display.start()

Preparamos o código que gera um vídeo com um episódio do jogo. O vídeo é armazenado na pasta video.

O código é quase uma cópia exata do método play_step () da classe Agent, sem a seleção de ação epsilon-greedy. Basta passarmos nossa observação para o agente e selecionarmos a ação com valor máximo. A única coisa nova aqui é o método render () no Environment, que é uma forma padrão no Gym de exibir a observação atual.

O loop principal no código passa a ação para o ambiente, conta a recompensa total e interrompe o loop quando o episódio termina. Após o episódio, mostra a recompensa total e a contagem de vezes que o agente executou a ação.

In [ ]:
# Taken (partially) from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/03_dqn_play.py


model='PongNoFrameskip-v4-best.dat'
record_folder="video"  
visualize=True

env = make_env(DEFAULT_ENV_NAME)
if record_folder:
        env = gym.wrappers.Monitor(env, record_folder, force=True)
net = DQN(env.observation_space.shape, env.action_space.n)
net.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage))

state = env.reset()
total_reward = 0.0

while True:
        start_ts = time.time()
        if visualize:
            env.render()
        state_v = torch.tensor(np.array([state], copy=False))
        q_vals = net(state_v).data.numpy()[0]
        action = np.argmax(q_vals)
        
        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
        if visualize:
            delta = 1/FPS - (time.time() - start_ts)
            if delta > 0:
                time.sleep(delta)
print("Total reward: %.2f" % total_reward)

if record_folder:
        env.close()